### Equal-Weight Index Fund

#### Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

#### Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

#### Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Now it's time to import these stocks to our Jupyter Notebook file.

We need the following information after **Parsing**:

* Weight in index of each stock
* Price of each stock

In [4]:
INDEXES = {'SP500': 'sp500', 
           'NASDAQ': 'nasdaq100', 
           'DOWJONES': 'dowjones'}

In [5]:
from bs4 import BeautifulSoup
from decimal import Decimal

def index_parse(index):
    
    url = f'https://www.slickcharts.com/{index}'

    request = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(request.text, "lxml")
    
    stats = soup.find('table',class_='table table-hover table-borderless table-sm')

    df = pd.read_html(str(stats))[0]
    df['% Chg'] = df['% Chg'].str.strip('()-%')
    df['% Chg'] = pd.to_numeric(df['% Chg'])
    df['Chg'] = pd.to_numeric(df['Chg'])

    return df

In [6]:
parse_data = index_parse(INDEXES['SP500'])
parse_data.head()

,#,Company,Symbol,Weight,Price,Chg,% Chg
0,1,Apple Inc.,AAPL,6.802701,173.28,0.21,0.12
1,2,Microsoft Corporation,MSFT,5.800789,310.20,0.00,0.00
2,3,Amazon.com Inc.,AMZN,3.564757,3239.00,-3.76,0.12
3,4,Tesla Inc,TSLA,2.127035,1045.10,-4.51,0.43
4,5,Alphabet Inc. Class A,GOOGL,2.113449,2785.00,-4.61,0.17


#### Define our deposit value and read Excel Portfolio file

In [7]:
depo = 0

In [8]:
while True:
    try:
        depo += float(input('Enter the value of your portfolio in USD '))
        break
    except Exception:
        print('Thats not a number! \n Try again:')

Enter the value of your portfolio in USD 1000


In [9]:
# read the file with all assets in portfolio
portfolio = pd.read_excel('portfolio.xlsx')

del portfolio['Unnamed: 0']

In [10]:
portfolio.head()

,asset,amount,price,weight,total_sum
0,AAPL,1,175,1,175


#### Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [11]:
class CalculateSupply:
    
    def __init__(self, user_portfolio, parse_data, depo):
        self.portfolio = user_portfolio
        self.parse_data = parse_data
        self.depo = depo
        
        self.total_investments = sum(self.portfolio['total_sum'])
        self.total_deposit = self.total_investments + self.depo
        self.total_assets_sum = self.portfolio.groupby('asset').agg({'total_sum': 'sum'})
        
        
    # parse elements from source
    def _parse_elem(self, elem):
        symbol = elem[1][2]
        official_weight = round(Decimal(elem[1][3] / 100), 5)
        official_price = elem[1][4]
        
        return symbol, official_weight, official_price
    
    # checking current allocation
    def _check_allocation(self, symbol):
        try:
            total_asset_sum = self.total_assets_sum.query(f"asset == '{symbol}'")['total_sum'][0]
        except Exception:
            total_asset_sum = 0

        weight = total_asset_sum / self.total_deposit
        
        return total_asset_sum, weight
    
    # calculating recommendation
    def _calculating_requirements(self, weight, official_weight, official_price, total_asset_sum): 
        
        required_weight = float(official_weight) - weight
        if required_weight > 0:
            required_allocation = self.total_deposit * required_weight
            required_investments = required_allocation - total_asset_sum
            if required_investments:
                required_amount = required_investments / official_price
                return required_investments, required_weight, required_amount
        return 0, 0, 0
                
    def run(self):
        # define df for recommendation
        my_columns = ['asset', 
                        'required_investments', 
                        'required_weight', 
                        'required_amount', 
                        'official_price']
        
        df = pd.DataFrame(columns=my_columns)
        
        for elem in self.parse_data.iterrows():
            # define necessary variables
            symbol, official_weight, official_price = self._parse_elem(elem)
            total_asset_sum, weight = self._check_allocation(symbol)
            required_investments, required_weight, required_amount = self._calculating_requirements(weight, 
                                                                                  official_weight, 
                                                                                  official_price, 
                                                                                  total_asset_sum)
            
            # adding recommendation
            df = df.append(pd.DataFrame([[
                                    symbol, 
                                    required_investments, 
                                    required_weight, 
                                    required_amount,
                                    official_price]], 
                                  columns=my_columns))


            self.depo -= float(required_investments)

        return df
        
calculator = CalculateSupply(portfolio, parse_data, depo)

In [13]:
# run our script
recommendation = calculator.run()
recommendation.sort_values('required_amount', ascending=False).head(10)

,asset,required_investments,required_weight,required_amount,official_price
0,MSFT,68.16175,0.05801,0.219735,310.20
0,BAC,10.4575,0.0089,0.218274,47.91
0,T,5.687,0.00484,0.209235,27.18
0,PFE,9.2825,0.0079,0.168681,55.03
0,CMCSA,6.9795,0.00594,0.13448,51.90
0,CSCO,7.7315,0.00658,0.126023,61.35
0,XOM,8.9065,0.00758,0.123925,71.87
0,VZ,6.6035,0.00562,0.123847,53.32
0,INTC,6.6505,0.00566,0.119185,55.80
0,F,2.914,0.00248,0.115681,25.19


#### Saving Our DataFrame to Excel

Saving our Excel file is very easy:

In [11]:
recommendation.to_excel('portfolio.xlsx')